<a href="https://colab.research.google.com/github/chasubeen/TalesRunnner/blob/doeun/Image_Captioning_BLIP_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image-Captioning-BLIP-Validation

## 코드 개요
BLIP 모델을 사용해 이미지로부터 캡션을 생성하고, 생성된 캡션을 한국어로 번역하여 지정된 CSV 파일에 저장 및 관리하는 코드.

## 작동 흐름
1. 환경 설정: 필요한 라이브러리와 파일 경로를 설정
2. 데이터 준비: 데이터 압축 해제 후 jpg 파일 필터링
3. 모델 및 프로세서 로드: BLIP 모델 및 프로세서 로드
4. 캡션 생성: BLIP 모델로 이미지 캡션 생성 및 GoogleTranslator로 한국어 번역 수행 후 결과물을 csv 파일로 저장
5. 작업 중단 시 대처: 캡션 생성 도중 작업 중단 시 대처 관련 코드
6. 최종 데이터 통계 확인: 데이터 전처리 완료 후 전체 데이터 개수(5000개) 확인

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration # 이미지 캡셔닝
from deep_translator import GoogleTranslator # 번역기

import os
import csv
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

## 2. 데이터 준비

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


**[데이터 경로 설정]**
- `zip_folder_path`: 데이터 zip 파일이 있는 경로
- `extraction_base_path`: 데이터 zip 파일을 압축 해제할 경로 (구글 드라이브가 아닌 가상 메모리에 업로드되도록 설정)

In [ ]:
### 데이터 압축 해제

# ZIP 파일들이 있는 폴더 경로 설정
zip_folder_path = '/content/drive/My Drive/7th-project/data/Validation'
extraction_base_path = '/content/data'

# ZIP 파일명을 저장할 리스트
zip_file_names = []

# ZIP 파일 순회
for file_name in os.listdir(zip_folder_path):
    # 파일이 ZIP 파일인지 확인
    if file_name.endswith('.zip'):
        zip_file_names.append(file_name)  # 파일명 추가
        zip_file_path = os.path.join(zip_folder_path, file_name)

        # 파일명과 동일한 디렉토리 생성
        extract_dir_name = os.path.splitext(file_name)[0]  # 확장자 제거한 이름
        extract_dir_path = os.path.join(extraction_base_path, extract_dir_name)
        os.makedirs(extract_dir_path, exist_ok=True)  # 디렉토리 생성

        # ZIP 파일 압축 해제
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_dir_path)
        print(f"압축 해제 완료: {file_name} -> {extract_dir_path}")

# ZIP 파일명 리스트 출력
print("처리된 ZIP 파일 리스트:", zip_file_names)

압축 해제 완료: VL_02T_자연탐구_03S_초등_고학년.zip -> /content/data/VL_02T_자연탐구_03S_초등_고학년
압축 해제 완료: VL_05T_신체운동_건강_03S_초등_고학년.zip -> /content/data/VL_05T_신체운동_건강_03S_초등_고학년
압축 해제 완료: VL_04T_예술경험_03S_초등_고학년.zip -> /content/data/VL_04T_예술경험_03S_초등_고학년
압축 해제 완료: VL_05T_신체운동_건강_02S_초등_저학년.zip -> /content/data/VL_05T_신체운동_건강_02S_초등_저학년
압축 해제 완료: VL_04T_예술경험_02S_초등_저학년.zip -> /content/data/VL_04T_예술경험_02S_초등_저학년
압축 해제 완료: VL_05T_신체운동_건강_01S_유아.zip -> /content/data/VL_05T_신체운동_건강_01S_유아
압축 해제 완료: VL_02T_자연탐구_02S_초등_저학년.zip -> /content/data/VL_02T_자연탐구_02S_초등_저학년
압축 해제 완료: VL_03T_사회관계_01S_유아.zip -> /content/data/VL_03T_사회관계_01S_유아
압축 해제 완료: VL_04T_예술경험_01S_유아.zip -> /content/data/VL_04T_예술경험_01S_유아
압축 해제 완료: VL_01T_의사소통_03S_초등_고학년.zip -> /content/d

**[파일 전처리]**

데이터 압축 해제 시 `.json`과 `.jpg` 파일이 함께 존재하므로, `.jpg` 파일만 남겨 처리 속도를 개선함

In [ ]:
### 파일 처리: .jpg 파일만 남기기

# 디렉토리 순회
for subdir, _, files in os.walk(extraction_base_path):
    total_files = len(files)  # 원래 전체 파일 개수
    jpg_files = 0  # .jpg 파일 개수 초기화

    for filename in files:
        file_path = os.path.join(subdir, filename)

        # .json 파일 삭제
        if filename.endswith('.json'):
            os.remove(file_path)  # 파일 삭제
        # .jpg 파일 개수 세기
        elif filename.endswith('.jpg'):
            jpg_files += 1

    # 결과 출력
    print(f"디렉토리: {subdir}")
    print(f".jpg 파일 개수: {jpg_files} / 원래 전체 파일 개수: {total_files}")
    print("-" * 50)

디렉토리: /content/data
.jpg 파일 개수: 0 / 원래 전체 파일 개수: 0
--------------------------------------------------
디렉토리: /content/data/VL_03T_사회관계_01S_유아
.jpg 파일 개수: 152 / 원래 전체 파일 개수: 304
--------------------------------------------------
디렉토리: /content/data/VL_05T_신체운동_건강_02S_초등_저학년
.jpg 파일 개수: 171 / 원래 전체 파일 개수: 342
--------------------------------------------------
디렉토리: /content/data/VL_03T_사회관계_03S_초등_고학년
.jpg 파일 개수: 442 / 원래 전체 파일 개수: 884
--------------------------------------------------
디렉토리: /content/data/VL_05T_신체운동_건강_03S_초등_고학년
.jpg 파일 개수: 225 / 원래 전체 파일 개수: 450
--------------------------------------------------
디렉토리: /content/data/VL_02T_자연탐구_03S_초등_고학년
.jpg 파일 개수: 163 / 원래 전체 파일 개수: 326
--------------------------------------------------
디렉토리: /content/data/VL_01T_의사소통_01S_유아
.jpg 파일 개수: 358 / 원래 전체 파일 개수: 716
--------------------------------------------------
디렉토리: /content/data/VL_04T_예술경험_03S_초드

## 3. 모델 및 프로세서 로드

- 모델: BLIP

In [ ]:
# BLIP 모델과 프로세서 로드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

**[캡션 생성 함수]**

- BLIP 모델을 사용해 입력 이미지에서 캡션 생성.
- 생성된 캡션을 `GoogleTranslator`을 사용하여 한국어로 번역.
- 출력: 이미지(`image`), 영어 캡션(`caption_eng`, 디버깅용), 한국어 캡션(`caption_kor`)

In [ ]:
### 캡션 생성 함수
def generate_korean_caption(image_path):

    # 이미지 로드
    image = Image.open(image_path).convert('RGB')

    # BLIP 입력 생성
    inputs = processor(image, return_tensors="pt")

    # 영어 캡션 생성
    outputs = model.generate(**inputs)
    caption_eng = processor.decode(outputs[0], skip_special_tokens=True)

    # 영어 캡션을 한국어로 번역
    caption_kor = GoogleTranslator(source='en', target='ko').translate(caption_eng)

    return image, caption_eng, caption_kor

**[이미지 시각화 함수]**

출력 확인용 시각화 함수

In [ ]:
### 이미지 시각화 함수
def display_image(image):
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')  # 축 제거
    plt.show()

## 4. 캡션 생성

In [ ]:
zip_file_names = []
for file_name in os.listdir(zip_folder_path):
    if file_name.endswith('.zip'):
        zip_file_names.append(os.path.splitext(file_name)[0])
extraction_base_path = '/content/data'

**[이미지 처리 및 CSV 업데이트 함수]**

- 입력된 이미지에서 캡션을 생성하고, 한국어로 번역.
- 이미지 파일명(`id`), 이미지 경로(`img_path`), 한국어 캡션(`caption`)을 포함한 데이터 구성.
- 기존 CSV 파일이 있으면 업데이트, 없으면 새로 생성하여 데이터 저장.

In [ ]:
# 이미지 처리 및 CSV 파일 생성/업데이트 함수
def process_images_and_update_csv(dir_path, file_name, csv_file_path):
    # 이미지 파일 전체 경로
    image_path = os.path.join(dir_path, file_name)

    # 캡션 생성
    image, caption_eng, caption_kor = generate_korean_caption(image_path)

    # 디버깅: 캡션 출력
    print(f"image_path: {image_path}, caption: {caption_kor}")

    # 파일명에서 확장자 제거
    file_id = os.path.splitext(file_name)[0]

    # 데이터 준비
    entry = {
        'file_name': file_id,
        'img_path': image_path,
        'caption': caption_kor
    }

    # CSV 파일 생성 또는 업데이트
    file_exists = os.path.exists(csv_file_path)
    with open(csv_file_path, mode='a' if file_exists else 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['id', 'img_path', 'caption'])

        # 헤더 작성 (파일이 없을 경우에만 작성)
        if not file_exists:
            writer.writeheader()

        # 데이터 작성
        writer.writerow({'id': entry['file_name'], 'img_path': entry['img_path'], 'caption': entry['caption']})


In [ ]:
# CSV 파일 경로 설정
csv_file_path = '/content/drive/My Drive/7th-project/data/img-captions-val.csv'

for zip_file_name in zip_file_names:
    dir_path = os.path.join(extraction_base_path, zip_file_name)

    # 디렉토리 내 파일 검색
    if os.path.exists(dir_path):
        file_names = os.listdir(dir_path)  # .jpg 파일 필터링했으므로 모든 파일 탐색
        total_images = len(file_names)  # 총 이미지 개수 계산

        # 파일 순회
        for idx, file_name in enumerate(file_names, start=1):
            print(f"[{idx}/{total_images}] {file_name} 처리 중...")
            process_images_and_update_csv(dir_path, file_name, csv_file_path)

    else:
        print(f"디렉토리가 존재하지 않습니다: {dir_path}")

    print(f">> {zip_file_name} 내의 이미지 처리 완료!\n")


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
image_path: /content/data/VL_05T_신체운동_건강_02S_초등_저학년/03_05T_02S_9791185721446_40148.jpg, caption: 아이가 공원에서 개와 놀고 있어요
[94/171] 03_05T_02S_9788927207054_16757.jpg 처리 중...
image_path: /content/data/VL_05T_신체운동_건강_02S_초등_저학년/03_05T_02S_9788927207054_16757.jpg, caption: 여자와 두 아이의 그림
[95/171] 03_05T_02S_9788927206521_8885.jpg 처리 중...
image_path: /content/data/VL_05T_신체운동_건강_02S_초등_저학년/03_05T_02S_9788927206521_8885.jpg, caption: 길을 걷고 있는 아이들의 무리
[96/171] 03_05T_02S_9788927206866_8407.jpg 처리 중...
image_path: /content/data/VL_05T_신체운동_건강_02S_초등_저학년/03_05T_02S_9788927206866_8407.jpg, caption: 주방에 있는 어린 소녀와 어린 소년의 그림
[97/171] 03_05T_02S_9788963835822_8267.jpg 처리 중...
image_path: /content/data/VL_05T_신체운동_건강_02S_초등_저학년/03_05T_02S_9788963835822_8267.jpg, caption: 비디오 게임을 하는 가족의 만화
[98/171] 03_05T_02S_9788963835839_7963.jpg 처리 중...
image_path: /content/data/VL_05T_신체우

## 5. 작업 중단 시 대처

### 디렉토리별 데이터 개수 계산

In [ ]:
### 패턴별 데이터 개수 계산

csv_file_path = '/content/drive/My Drive/7th-project/data/img-captions-val.csv'

# 특정 패턴 리스트 정의
patterns = [
    f"{i:02d}T_{j:02d}S"
    for i in range(1, 6)  # 01T ~ 05T
    for j in range(1, 4)  # 01S ~ 03S
]

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 각 패턴별 데이터 개수 계산
pattern_counts = {pattern: df['id'].str.contains(pattern).sum() for pattern in patterns}

# 결과 출력
for pattern, count in pattern_counts.items():
    print(f"{pattern}: {count}개")


01T_01S: 358개
01T_02S: 0개
01T_03S: 433개
02T_01S: 0개
02T_02S: 204개
02T_03S: 163개
03T_01S: 152개
03T_02S: 0개
03T_03S: 442개
04T_01S: 325개
04T_02S: 191개
04T_03S: 48개
05T_01S: 243개
05T_02S: 171개
05T_03S: 225개


### (필요 시) 데이터 삭제

In [ ]:
### 실행되다 만 디렉토리의 데이터 삭제

import pandas as pd

# CSV 파일 경로
csv_file_path = '/content/drive/My Drive/7th-project/data/img-captions-val.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 삭제 대상 데이터 필터링
to_delete = df['id'].str.contains('03T_02S', na=False)  # '03T_02S'를 포함한 데이터
deleted_count = to_delete.sum()  # 삭제된 데이터 개수

# 삭제 수행
df_filtered = df[~to_delete]  # 삭제 대상이 아닌 데이터만 필터링

# CSV 파일 저장 (필요 시 덮어쓰기)
output_file_path = '/content/drive/My Drive/7th-project/data/img-captions-val.csv'
df_filtered.to_csv(output_file_path, index=False)

# 결과 출력
print(f"'03T_02S'를 포함한 데이터 {deleted_count}개 삭제 완료")
print(f"새로운 CSV 파일이 저장되었습니다: {output_file_path}")

'03T_02S'를 포함한 데이터 669개 삭제 완료
새로운 CSV 파일이 저장되었습니다: /content/drive/My Drive/7th-project/data/img-captions-val.csv


In [ ]:
### csv 파일에서 마지막 n개의 열 삭제

import pandas as pd

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/7th-project/data/img-captions-val.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 마지막 169개의 데이터 삭제
df = df[:-169]  # 끝에서 169개의 행을 제외한 데이터프레임 생성

# 수정된 데이터 저장
df.to_csv(csv_file_path, index=False)

print(f"CSV 파일에서 마지막 169개의 데이터를 삭제하고 저장했습니다.")


### 지정된 디렉토리의 이미지 캡션 생성

In [ ]:
pending_zip_file_names = ['VL_03T_사회관계_02S_초등_저학년', 'VL_01T_의사소통_02S_초등_저학년', 'VL_02T_자연탐구_01S_유아']

csv_file_path = '/content/drive/My Drive/7th-project/data/img-captions-val.csv'

for zip_file_name in pending_zip_file_names:
    dir_path = os.path.join(extraction_base_path, zip_file_name)

    # 디렉토리 내 파일 검색
    if os.path.exists(dir_path):
        file_names = os.listdir(dir_path)  # .jpg 파일 필터링했으므로 모든 파일 탐색
        total_images = len(file_names)  # 총 이미지 개수 계산

        # 파일 순회
        for idx, file_name in enumerate(file_names, start=1):
            print(f"[{idx}/{total_images}] {file_name} 처리 중...")
            process_images_and_update_csv(dir_path, file_name, csv_file_path)

    else:
        print(f"디렉토리가 존재하지 않습니다: {dir_path}")

    print(f">> {zip_file_name} 내의 이미지 처리 완료!\n")

[1/670] 03_03T_02S_9788954339551_58586.jpg 처리 중...
image_path: /content/data/VL_03T_사회관계_02S_초등_저학년/03_03T_02S_9788954339551_58586.jpg, caption: 동굴에서 두 남자가 악수하는 그림
[2/670] 03_03T_02S_9791165140847_55935.jpg 처리 중...
image_path: /content/data/VL_03T_사회관계_02S_초등_저학년/03_03T_02S_9791165140847_55935.jpg, caption: 비 속에 있는 사람들의 그룹을 그린 수채화 그림
[3/670] 03_03T_02S_9791165430450_67086.jpg 처리 중...
image_path: /content/data/VL_03T_사회관계_02S_초등_저학년/03_03T_02S_9791165430450_67086.jpg, caption: 꽃이 놓인 테이블에 앉아 있는 여성과 두 아이
[4/670] 03_03T_02S_9791165954697_32846.jpg 처리 중...
image_path: /content/data/VL_03T_사회관계_02S_초등_저학년/03_03T_02S_9791165954697_32846.jpg, caption: 차 안에 있는 두 소녀의 그림
[5/670] 03_03T_02S_9788954339247_33039.jpg 처리 중...
image_path: /content/data/VL_03T_사회관계_02S_초등_저학년/03_03T_02S_9788954339247_33039.jpg, caption: 전통 의상을 입은 두 남자
[6/670] 03_03T_02S_9788954339537_36197.jpg 처리 중...
image_path: /content/data/VL_03T_사회관계_02S_초등_저

## 6. 최종 데이터 통계 확인

In [ ]:
### 데이터 수 확인 코드
import pandas as pd

# CSV 파일 경로
csv_file_path = '/content/drive/MyDrive/7th-project/data/captions_val.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path)

# 데이터 개수 확인
row_count = len(df)
print(f"CSV 파일의 데이터 개수: {row_count}개")

CSV 파일의 데이터 개수: 5000개
